In [15]:
import numpy as np
import sympy as sp
import re
import argparse

import torch

# parser = argparse.ArgumentParser()
# parser.add_argument("n", help="Enter the dimension of the system", type=int)
# args = parser.parse_args()
#
# n = args.n

"""Define the dimensionality of the system"""
n = 4

"""Define the variable for which you want the auto-spectrum"""
i = 0

In [16]:
"""Define the matrix J which is n*n and elements like [a11, 112; a21, a22]"""
J = sp.Matrix([[sp.symbols('a%d%d' % (k+1, j+1)) for j in range(n)] for k in range(n)])

In [17]:
"""Define the noise variables"""
l = [sp.Symbol('l%d' % (j)) for j in range(1, n+1)] # this is the noise variable
s = [sp.Symbol('s%d' % (j)) for j in range(1, n+1)] # this is the sigma variable

"""Define the O and P (submatrix of O) matrices"""
O = [sp.Symbol('O%d%d' % (j, i+1)) for j in range(1, n+1)]
O_prime = [sp.Symbol('O\'%d%d' % (j, i+1)) for j in range(1, n+1)]

"""Denominator"""
J = sp.symbols('J')

In [18]:
"""Define the functions"""
def power(x, n):
    return sp.sympify(x**n)

"""Define the trace function"""
def Tr(x):
    return sp.Function('Tr')(x)

In [19]:
"""Define Bell polynomial matrix"""
def bell_inp(X, kappa, k):
    """
    Constructs the traces of the matrix input X
    """
    if k != 0:
        x = sp.zeros(k, 1)
        for i in range(0, k):
            x[i] = - Tr(power(X, kappa * (i + 1)))
        return sp.ImmutableMatrix(x)
    else:
        return sp.ImmutableMatrix(sp.ones(1))

def comp_bell(x):
    k = x.shape[0]
    B_k = sp.zeros(k, k)
    for i in range(0, k):
        for j in range(0, k):
            if j - i < -1:
                B_k[i, j] = 0
            elif j - i == -1:
                B_k[i, j] = -i
            else:
                B_k[i, j] = x[j - i]
    return sp.det(B_k)

In [20]:
"""Additional functions"""
def d(A, alpha, n):
    """
    d(w; A) = ||A+iwI||^2
    Returns the coefficient of w^{2*alpha}
    """
    return (-1) ** abs(n - alpha) * comp_bell(bell_inp(A, 2, n - alpha)) / sp.factorial(n - alpha)

def g_2(A, B, alpha, n):
    """
    g2 = Imaginary{2w Conjugate{|A+iwI|}|B+iwI|}
    Returns the coefficient of the power of 2*alpha. See SI for details.
    """
    temp = sp.Rational("0")
    for j in range(n + 1):
        k = 2 * alpha - 1 - j
        if k <= n - 1 and k >= 0:
            coeff = 2 * (-1) ** abs(alpha - j - 1) / (sp.factorial(n - j) * sp.factorial(n - k - 1))
            temp += coeff * comp_bell(bell_inp(A, 1, n - j)) * comp_bell(bell_inp(B, 1, n - k - 1))
    return temp

In [21]:
def f(A, B, alpha, n):
    """
    See SI for definition.
    """
    return d(A, alpha, n-1) + d(B, alpha - 1, n-2) + g_2(A, B, alpha, n-1)

In [22]:
def p_alpha(i, alpha, l, s, O, P, n):
    """
    This function returns the coefficient of the numerator of the auto-spectrum
    of the i-th variable.
    :param i: the index of the variable for which the auto-spectra is desired.
    :param alpha: power of omega
    :return: the value of the coefficient of omega^2alpha.
    """
    temp = sp.Rational("0")
    for m in range(n):
        if not m == i:
            temp += s[m] ** 2 * l[m] ** 2 * f(O[m], P[m], alpha, n)
        else:
            temp += s[m] ** 2 * l[m] ** 2 * d(O[m], alpha, n-1)
    return temp

def q_alpha(J, alpha, n):
    """
    This function returns the coefficient of the denominator of the spectrum.
    Note that the denominator is the same for all the variables.
    :param alpha: power of omega
    :return: the value of the coefficient of omega^2alpha.
    """
    return d(J, alpha, n)

In [23]:
"""Coefficients of the denominator"""
alpha = 0
den_coeff = sp.together(sp.simplify(q_alpha(J, alpha, n)))

denm_str = sp.latex(den_coeff, mode='equation')
denm_str = denm_str.replace("J", "\mathbf{J}")
denm_str = denm_str.replace("\operatorname{Tr}", "\Tr")
print(denm_str)

\begin{equation}\frac{\Tr^{4}{\left(\mathbf{J}^{2} \right)} - 6 \Tr^{2}{\left(\mathbf{J}^{2} \right)} \Tr{\left(\mathbf{J}^{4} \right)} + 8 \Tr{\left(\mathbf{J}^{2} \right)} \Tr{\left(\mathbf{J}^{6} \right)} + 3 \Tr^{2}{\left(\mathbf{J}^{4} \right)} - 6 \Tr{\left(\mathbf{J}^{8} \right)}}{24}\end{equation}


In [28]:
"""Coefficients of the numerator"""
alpha = 0
num_coeff = sp.together(sp.simplify(p_alpha(i, alpha, l, s, O, O_prime, n)))
num_str = sp.latex(num_coeff, mode='equation')
print(num_coeff)

(l1**2*s1**2*(Tr(O11**2)**3 - 3*Tr(O11**2)*Tr(O11**4) + 2*Tr(O11**6)) + l2**2*s2**2*(Tr(O'21**2)**3 - 3*Tr(O'21**2)*Tr(O'21**4) + 2*Tr(O'21**6) + Tr(O21**2)**3 - 3*Tr(O21**2)*Tr(O21**4) + 2*Tr(O21**6)) + l3**2*s3**2*(Tr(O'31**2)**3 - 3*Tr(O'31**2)*Tr(O'31**4) + 2*Tr(O'31**6) + Tr(O31**2)**3 - 3*Tr(O31**2)*Tr(O31**4) + 2*Tr(O31**6)) + l4**2*s4**2*(Tr(O'41**2)**3 - 3*Tr(O'41**2)*Tr(O'41**4) + 2*Tr(O'41**6) + Tr(O41**2)**3 - 3*Tr(O41**2)*Tr(O41**4) + 2*Tr(O41**6)))/6


In [29]:
"""Processing the string for printing"""
pattern = r"O'(\d{2})"
num_str = re.sub(pattern, r"O^\\prime_{\1}", num_str)
num_str = num_str.replace("O", "\mathbf{O}")
num_str = num_str.replace("s_", "\sigma_")
num_str = num_str.replace("\operatorname{Tr}", "\Tr")
num_str = num_str.replace("\left", "")
num_str = num_str.replace("\\right", "")
print(num_str)

\begin{equation}\frac{l_{1}^{2} \sigma_{1}^{2} (\Tr^{3}{(\mathbf{O}_{11}^{2} )} - 3 \Tr{(\mathbf{O}_{11}^{2} )} \Tr{(\mathbf{O}_{11}^{4} )} + 2 \Tr{(\mathbf{O}_{11}^{6} )}) + l_{2}^{2} \sigma_{2}^{2} (\Tr^{3}{(\mathbf{O}^\prime_{21}^{2} )} - 3 \Tr{(\mathbf{O}^\prime_{21}^{2} )} \Tr{(\mathbf{O}^\prime_{21}^{4} )} + 2 \Tr{(\mathbf{O}^\prime_{21}^{6} )} + \Tr^{3}{(\mathbf{O}_{21}^{2} )} - 3 \Tr{(\mathbf{O}_{21}^{2} )} \Tr{(\mathbf{O}_{21}^{4} )} + 2 \Tr{(\mathbf{O}_{21}^{6} )}) + l_{3}^{2} \sigma_{3}^{2} (\Tr^{3}{(\mathbf{O}^\prime_{31}^{2} )} - 3 \Tr{(\mathbf{O}^\prime_{31}^{2} )} \Tr{(\mathbf{O}^\prime_{31}^{4} )} + 2 \Tr{(\mathbf{O}^\prime_{31}^{6} )} + \Tr^{3}{(\mathbf{O}_{31}^{2} )} - 3 \Tr{(\mathbf{O}_{31}^{2} )} \Tr{(\mathbf{O}_{31}^{4} )} + 2 \Tr{(\mathbf{O}_{31}^{6} )}) + l_{4}^{2} \sigma_{4}^{2} (\Tr^{3}{(\mathbf{O}^\prime_{41}^{2} )} - 3 \Tr{(\mathbf{O}^\prime_{41}^{2} )} \Tr{(\mathbf{O}^\prime_{41}^{4} )} + 2 \Tr{(\mathbf{O}^\prime_{41}^{6} )} + \Tr^{3}{(\mathbf{O}_{41}^{2} )}

In [20]:
from spectrum_general.symbolic_spectrum import symbolic_PSD

In [55]:
n = 4
J = sp.symbols('J')
l = [sp.Symbol('l_%d' % (j)) for j in range(n)]
L = sp.diag(*l)
s = [sp.Symbol('s_%d' % (j)) for j in range(n)]
S = sp.diag(*s)
print(L[0])

l_0


In [23]:
model = symbolic_PSD(J, L, S)

AttributeError: 'Symbol' object has no attribute 'tolist'

In [48]:
n = 4
O = [[sp.Symbol('O_{}{}'.format(i, j)) for j in range(n)] for i in range(n)]
# extract the second column and store it as a list

print(O_prime)

[O_01, O_11, O_21, O_31]


In [40]:
model.l

NameError: name 'model' is not defined